In [2]:
%pip install transformers accelerate pdf2image
%pip install  huggingface_hub
!sudo apt-get update
!sudo apt-get install -y poppler-utils

  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached accelerate-1.2.1-py3-none-any.whl (336 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1199 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:

In [3]:
%pip install vllm

  Using cached vllm-0.6.6.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (11 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached blake3-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached protobuf-5.29.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached openai-1.59.3-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached prometheus_fastapi_instrumentator-7.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached tiktok

In [ ]:
from huggingface_hub import login
login(token="")

In [4]:
from pdf2image import convert_from_path
pdf_path = "/home/1735246990262LuciaMortayaLopezMedicalRecords1.pdf"

In [6]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from vllm.assets.image import ImageAsset
from vllm.utils import FlexibleArgumentParser
from datetime import datetime
from zoneinfo import ZoneInfo
import time

# Qwen2-VL Model Setup
def run_qwen2_vl():
    model_name = "Qwen/Qwen2-VL-7B-Instruct"

    llm = LLM(
        model=model_name,
        max_num_seqs=5,
    )

    prompt = ("<|im_start|>system\nYou are a helpful assistant. Extract all data from the image provided. Return the result in a detailed markdown format. Don't miss any information.<|im_end|>\n"
              "<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>\n"
              "<|im_end|>\n<|im_start|>assistant\n")
    stop_token_ids = None
    return llm, prompt, stop_token_ids

# Prepare Image and Question Input
def get_image_input(image):
    return {
        "data": image,
    }

# Main Inference Function
def main():
    # Model Setup
    images = convert_from_path(pdf_path)

    llm, prompt, stop_token_ids = run_qwen2_vl()

     # Sampling Parameters
    sampling_params = SamplingParams(temperature=0.2,
                                     max_tokens=500,
                                     stop_token_ids=stop_token_ids)

    mark_down = ""
    total_sec = 0
    
    for index, image in enumerate(images):
        start_time = time.time()
         # Prepare Inputs
        inputs = {
            "prompt": prompt,
            "multi_modal_data": {
                "image": image
            },
        }
        # Generate Output
        outputs = llm.generate(inputs, sampling_params=sampling_params)

        end_time = time.time()  
        elapsed_time = end_time - start_time
        total_sec += elapsed_time
        print(f"time taken {elapsed_time:.2f}")
        for o in outputs:
            generated_text = o.outputs[0].text
            mark_down += generated_text
            mark_down += "\n"
    print(mark_down)
    print(f"total_sec ----- {total_sec}")
    

main()


INFO 01-07 07:06:17 config.py:510] This model supports multiple tasks: {'classify', 'generate', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 01-07 07:06:17 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='Qwen/Qwen2-VL-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2-

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 01-07 07:06:22 model_runner.py:1099] Loading model weights took 15.5008 GB
INFO 01-07 07:06:32 worker.py:241] Memory profiling takes 10.06 seconds
INFO 01-07 07:06:32 worker.py:241] the current vLLM instance can use total_gpu_memory (39.39GiB) x gpu_memory_utilization (0.90) = 35.45GiB
INFO 01-07 07:06:32 worker.py:241] model weights take 15.50GiB; non_torch_memory takes -0.27GiB; PyTorch activation peak memory takes 4.74GiB; the rest of the memory reserved for KV Cache is 15.48GiB.
INFO 01-07 07:06:33 gpu_executor.py:76] # GPU blocks: 18121, # CPU blocks: 4681
INFO 01-07 07:06:33 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 8.85x
INFO 01-07 07:06:33 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_uti

Capturing CUDA graph shapes: 100%|██████████| 4/4 [00:01<00:00,  2.17it/s]

INFO 01-07 07:06:34 model_runner.py:1535] Graph capturing finished in 2 secs, took 0.02 GiB
INFO 01-07 07:06:34 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 12.22 seconds


...

time taken 8.84


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.51s/it, est. speed input: 571.41 toks/s, output: 58.74 toks/s]


time taken 8.84


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.55s/it, est. speed input: 568.97 toks/s, output: 58.49 toks/s]

time taken 8.94
# Defendant's Advocacy Statement
## Panel Qualified Medical Examination Letter

---

**Address for Cover Letter & Records:**
Dr. Purab Viswanath
5158 Clareton Dr, Ste 2158
Agoura Hills, CA 91376

**Address for QME Evaluation:**
27620 Lamdau Blvd, Ste 1
Cathedral City, CA 92234
(323) 487-2739

---

**RE:**
Morataya Lopez, Lucia Carolina v. Hospitality Staffing Solutions, LLC

**WCAB Number:** ADJ19331655
**Claim Number:** FOL2877
**Date(s) of Injury:** 06/04/2023

**PQME Appointment:** December 10, 2024 at 6:00 PM

---

**Dear Dr. Viswanath:**

Thank you for agreeing to see the applicant concerning the above-captioned claim in the capacity of an Orthopedic Qualified Medical Examiner in this case. Applicant is scheduled for her initial QME appointment on December 10, 2024 at 6:00 PM in your office located in Cathedral City, CA.

---

**CASE FACTS:**

This matter involves a 48-year-old cleaner alleging an injury to the right shoulder as a result of throwing out the trash. 